# Lecture 10: CNN
 
## Structure
CNN (Convolutional Neural Network)

Input -> Feature Extraction (convolution + subsampling) -> Classification (fully connected, Dense Net) -> Output

## Convolution

Cut a small "Patch" from the whole data and do operations to get one value.

Example) cut 3x3x1 input data with 2x2x1 filter, stride 1x1 :=> 2x2x1 data is generated 


$$
Example =  \begin{bmatrix}
1 & 2 & 3\\
4 & 5 & 6\\
7 & 8 & 9 
\end{bmatrix} 
,
Filter = \begin{bmatrix}
0.1 & 0.5 \\
0.3 & 0.4
\end{bmatrix}
$$

$$
Result = \begin{bmatrix}
4.3 & 5.6 \\
8.2 & 9.5
\end{bmatrix}
$$


$$ ex) 1 * 0.1  + 2 * 0.5 + 4 * 0.3 + 5 * 0.4 = 4.3 , 2 * 0.1  + 3 * 0.5 + 5 * 0.3 + 6 * 0.4 = 5.6 , ... $$

Usually, dot product $$ w^T x +b $$ is used.


## ConvNet

For Example, Operating with a 32px * 32px image:

Each pixel has a RGB Data, so it the input data size is 32 * 32 *3.
If we had 6 5 * 5 * 3 filters, we would get 6 seperate 28 * 28 activation maps. (28 * 28 * 6 data)

ConvNet is a Sequence of Convolutional Layers, interspersed with activation functions.

## Pooling

Max Pooling : Choose the maximum data from a certain filter size to compress the data.

Example: Max pooling with 2x2 filter and stride 2

$$
Example =  \begin{bmatrix}
1 & 1 & 2 & 4\\
5 & 6 & 7 & 8\\
3 & 2 & 1 & 0\\
1 & 2 & 3 & 4 
\end{bmatrix} 
$$
$$
Result = \begin{bmatrix}
6 & 8 \\
3 & 4
\end{bmatrix}
$$

Average Pooling : Calculate the average data from a certain filter size to compress the data.

Example: Average pooling with 2x2 filter and stride 2

$$
Example =  \begin{bmatrix}
1 & 1 & 2 & 4\\
5 & 6 & 7 & 8\\
3 & 2 & 1 & 0\\
1 & 2 & 3 & 4 
\end{bmatrix} 
$$
$$
Result = \begin{bmatrix}
3.25 & 5.25 \\
2 & 2
\end{bmatrix}
$$

## Simple CNN

In CNN, Features are Localy Connected!



In [4]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torchvision import datasets, transforms
from torch.autograd import Variable


# Training Settings
batch_size=64
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())
# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Model Design
class Net(nn.Module):
    def __init__(self):
        super (Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)

        self.fc = nn.Linear(320 , 10) # 320 -> 10

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1) # Flatten the tnesor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max log-porbability
        pred = torch.max(output.data, 1)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

for epoch in range(1, 10):
    train(epoch)
    test()


/var/folders/j6/9s4ycw_d3q567rb82x3f01l40000gn/T/ipykernel_4185/1610885877.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.311669
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.291767
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.282655
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.253082
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.242240
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.186744
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.186741
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.129659
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 1.998003
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 1.854017
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 1.616081
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 1.318685
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 1.103563
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 0.916893
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 0.808342
Train Epoch: 1 | Batch Status: 9600/60

/var/folders/j6/9s4ycw_d3q567rb82x3f01l40000gn/T/ipykernel_4185/1610885877.py:71: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)


Test set: Average loss: 0.0030, Accuracy: 9435/10000 (94%)
Train Epoch: 2 | Batch Status: 0/60000 (0%) | Loss: 0.240005
Train Epoch: 2 | Batch Status: 640/60000 (1%) | Loss: 0.150401
Train Epoch: 2 | Batch Status: 1280/60000 (2%) | Loss: 0.220948
Train Epoch: 2 | Batch Status: 1920/60000 (3%) | Loss: 0.233398
Train Epoch: 2 | Batch Status: 2560/60000 (4%) | Loss: 0.079008
Train Epoch: 2 | Batch Status: 3200/60000 (5%) | Loss: 0.204159
Train Epoch: 2 | Batch Status: 3840/60000 (6%) | Loss: 0.167351
Train Epoch: 2 | Batch Status: 4480/60000 (7%) | Loss: 0.207240
Train Epoch: 2 | Batch Status: 5120/60000 (9%) | Loss: 0.150037
Train Epoch: 2 | Batch Status: 5760/60000 (10%) | Loss: 0.368356
Train Epoch: 2 | Batch Status: 6400/60000 (11%) | Loss: 0.169885
Train Epoch: 2 | Batch Status: 7040/60000 (12%) | Loss: 0.165576
Train Epoch: 2 | Batch Status: 7680/60000 (13%) | Loss: 0.233635
Train Epoch: 2 | Batch Status: 8320/60000 (14%) | Loss: 0.175475
Train Epoch: 2 | Batch Status: 8960/60000 (1